## Strategies
### Word focused
If the vocabulary is what you are looking at, the next step is to remove the punctuation and the stopwords. 

With the following code, we are going to generate a list of words used in the `Body` column. It won't be extremely useful for the classification task but comparing vocabulary can be very revealing (given the right subset: spam vs. regular emails,etc.).

Once we have a list of words, we can start the vocabulary analysis by coding a counter or using `FreqDist` from the `nltk` library for example.

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv', index_col='Id')
df

In [ ]:
for text in df['Body']:
    tokens = word_tokenize(text)
    # converts to lower case
    tokens = [tok.lower() for tok in tokens]
    # removes the stopwords
    words = [word for word in tokens if word not in stop_words]

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

stop_words = list(string.punctuation)
stop_words += stopwords.words('english')

In [1]:
def toklowstop(text):
    tokens = word_tokenize(text)
    # converts to lower case
    tokens = [tok.lower() for tok in tokens]
    # removes the stopwords
    words = [word for word in tokens if word not in stop_words]
    return words

In [ ]:
df['Body'].apply(lambda x: toklowstop(x))

In [ ]:
FreqDist(words).plot(50)
plt.show()

## Wordcloud
Wordclouds are to text data what pie charts are to numerical data. They are at best confusing and at worst useless. But it can be generate a nice picture for your article header :)

In [ ]:
from wordcloud import WordCloud
wordcloud = WordCloud(width = 3000,
                      height = 2000,
                      stopwords = stop_words)

wordcloud.generate(" ".join(words))

plt.imshow(wordcloud)
plt.axis('off')
plt.show()